In [ ]:
from pathlib import Path

In [ ]:
DATA = Path('../tests/data').resolve()

# Sanitize

This is the process of curating a morphological file. It currently:

- removes zero-length segments
- raises if the morphology has no soma
- raises if the morphology has negative diameters

Note: more functionality may be added in the future

## Sanitize a single file

In [ ]:
from neuror.sanitize import sanitize

sanitize(Path(DATA, 'simple-with-duplicates.asc'), 'data/output/sanitized.swc')

## Sanitize all files in a folder

In [ ]:
from neuror.sanitize import sanitize_all

sanitize_all(DATA, 'data/output/sanitized')

# Cut plane repair

The cut plane repair aims at regrowing part of a morphologies that have been cut out
when the cell has been experimentally sliced.

## Repair a single file

In [ ]:
from neuror.main import repair

repair(DATA / 'Neuron_slice.h5', 'data/output/repaired.h5')


## Repair all files in a folder

In [ ]:
from neuror.full import repair_all

output = Path('data/output/repaired-folder')
output.mkdir(exist_ok=True)

repair_all(DATA / '../../examples/input-repair', output)

# Unravelling

Unravelling is the action of "stretching" the cell that has been shrunk because of the dehydratation caused by the slicing.

## Unravel a single file

In [ ]:
from neuror.unravel import unravel

unravelled_neuron, mapping = unravel(DATA / 'Neuron_slice.h5')
unravelled_neuron.write('data/output/unravelled.h5')

# mapping contains the coordinates of each points of the morphology before/after unravelling
mapping

## Unravel all morphologies in a folder

The following `unravel_all` function not only perform the unravelling of all the morphologies in the folder but also update their cut planes.

Because the unravelling distords morphologies, cut planes must be computed before correcting it.
`unravel_all` takes care of updating the position of the 'cut points' (points close to the cut plane) after they have been unravelled.
Note: the planes can be computed with the CLI: neuror cut-plane compute folder

The argument 'raw_planes' is a folder that must contain JSON files with cut planes for all morphology in the input folder. The name of the files should be the same as the morphology they refer to, but with the JSON extension.


In [ ]:
from neuror.unravel import unravel_all

output = Path('data/output/unravelled-folder')
output.mkdir(exist_ok=True)

input_path = DATA / 'input-unravel-all'
raw_planes = input_path / 'raw-planes/'

unravel_planes = output / 'unravelled-planes'
unravel_planes.mkdir(exist_ok=True)

unravel_all(input_path, output, 
            raw_planes, unravel_planes)